# Presentación de la empresa, organización o problema específico  

#### De acuerdo a la Organización Mundial de la Salud (OMS), ACV es la segunda causa de muerte a nivel global, responsable de aproximadamente 11% de las muertes totales.
#### Este set de datos es utilizado para predecir si un paciente es propenso a tener un ACV basado en los parametros como género, edad, enfermedades varias, y estado de fumador. Cada fila en los datos provee información relevante sobre el paciente.


#### Preguntas y objetivos de la investigación.

.¿Cuál es el género más propenso a tener un acv?  
.¿Cuál es el grado de influencia de ser fumador?  
.¿Existe relación entre el índice de masa corporal y el nivel de glucosa para contraer un acv?  
.¿Qué tanto influye el tipo de trabajo en la posibilidad de tener un acv?  
.¿Aumentan las probabilidades de tener un acv a mayor edad?    
.¿Tener hipertensión influye a la hora de tener un acv?  // Se cambió la pregunta en base a la corrección de Maxi, no tenemos niveles de hipertensión, sólo veradero o falso
.¿Tiene relación el estado marital con tener un acv?  


El objetivo de la investigación es determinar cuales son las variables mas influyentes a la hora de provocar un acv. Utilizando un modelo de predicción vamos a poder determinar si una persona es mas propensa a tener un acv.





#### 3. Conformación del equipo de trabajo. 
Martin Marino  
Leandro Bruzzo  

# Indicación de la fuente del dataset y los criterios de selección (Data Acquisition)
#### El dataset fue obtenido de la plataforma Kaggle desde el siguiente link : https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset

#### Información sobre el data set:
- Creador: fedesoriano - https://www.kaggle.com/fedesoriano

- Información de los atributos: 
1) id: identificador único  
2) gender: "Male", "Female" o "Other" (Masculino, Femenino u Otro)  
3) age: Edad del paciente  
4) hypertension: 0 si el paciente no tiene hipertensión , 1 si el paciente tiene hipertensión  
5) heart_disease: 0 si el paciente no tiene enfermedades del corazón, 1 si el paciente tiene enfermedades del corazón  
6) ever_married: "No" o "Yes" (No o Sí)  
7) work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed" (Niño, trabajo en el sector público, nunca trabajó, privado o monotributista)  
8) Residence_type: "Rural" or "Urban"   
9) avg_glucose_level: nivel promedio de glucosa en la sangre  
10) bmi: Índice de masa corporal  
11) smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"* (Fumador frecuente, nunca fumó, fuma, desconocido)  
12) stroke: 1 si el paciente tuvo un ACV 0 si no lo tuvo (Variable Target)  
*Nota: 'Uknown' en smoking_status indica que no se registró un estado del paciente.

Información sobre Kaggle:
Kaggle, una subsidiaria de Google LLC, es una comunidad en línea de científicos de datos y profesionales del aprendizaje automático.

In [ ]:
#Librerías neceasrias para el correcto funcionamiento del jupyter (descomentar para ejecutar por primera vez y luego volver a comentarlo)

# import sys
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install seaborn
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install pandas_profiling
# !{sys.executable} -m pip install plotly.express
# !{sys.executable} -m pip install ipywidgets
# !{sys.executable} -m pip install nbformat
# !{sys.executable} -m pip install sklearn

#### Importamos todas las librerias de visualización, modelos de ML y validación

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling
import plotly.express as px

from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import roc_curve

from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv('healthcare-dataset-stroke-data.csv')
df.head()

In [ ]:
df.shape

In [ ]:
#Creamos un dataframe con las columnas work_type y residence_type en booleano (Es decir, 0 y 1) para ser utilizados en la matriz de correlación

df.rename(columns = {'Residence_type':'residence_type'}, inplace = True)
df_2 = df #Realizamos una copia del dataframe original para la matriz de correlación
#Reemplazamos los valores Yes y No de ever_married con 1 y 0
df_2['ever_married'].replace(
                to_replace=['Yes', 'No'], 
                value=[1,0,], 
                inplace=True, 
)
df_2['residence_type'].replace(
                to_replace=['Urban', 'Rural'], 
                value=[0,1,], 
                inplace=True, 
)

df_2

In [ ]:
df_2.describe().T

#### EDA Utilizando la librería pandas_profiling

In [ ]:
df_2.profile_report(title='Data Profiling')

#### Con el siguiente gráfico de torta podemos observar el gran desbalanceo de nuestro set de datos. Esto será crucial a la hora de preparar nuestros algoritmos, ya que un desbalance tan marcado puede ocasionar una gran pérdida de precisión y será necesario el uso de técnicas específicas para tratar dicho desbalanceo.


In [ ]:
df_pie = df.copy()
df_pie['stroke'].replace(
                to_replace=[0, 1], 
                value=["No","Yes"], 
                inplace=True)
fig = px.pie(df_pie, names="stroke",values='id', hole=.35, title='Stroke Distribution')
fig.show()

In [ ]:
df.stroke.value_counts()

#### Filtros y limpieza del dataset a utilizar para los modelos de ML

In [ ]:
#Vemos los valores unicos para la columna de smoking status y posteriormente excluimos los desconocidos
df['smoking_status'].unique()

#### En smoking_status se puede observar que tenemos valores "Unknown", se realizará un conteo de los mismos para observar el impacto de su eliminación, ya que dicho estado no nos aporta nada en el análisis.

In [ ]:
df.smoking_status.value_counts()

#### Unknown tiene una cantidad de registros de 1544, aproximadamente el 30% de los datos. Debido a que no podemos asumir si esa persona fuma o no, se tomará la decisión de eliminarlos. 

In [ ]:
#Eliminar los datos unknown de la columna smoking_status

df['smoking_status'].values
df = df[df.smoking_status != 'Unknown']
df.smoking_status.value_counts()

#### BMI tiene 140 valores nulos, los mismos podrían ser eliminados pero debido a que el dataset perdió gran cantidad de datos al eliminar los registros con smoking_status = Unknown, se precederá a utilizar un promedio para los mismos.


In [ ]:
for i in df.columns[df.isnull().any(axis=0)]:   
    df[i].fillna(df[i].mean(),inplace=True)
df.info()

#### Como se pudo observar en la línea anterior, se han completados los valores NaN de la columna BMI con el valor promedio de los mismos

#### En la columna gender podemos observar que hay un único registro con el género other, por motivos de espacio de memoria el mismo será eliminado ya que al preparar el dataset para utilizar algoritmos de clasificación empleando data dummies, se creará una columna extra en consecuencia de este único valor.

In [ ]:
df.gender.value_counts()

In [ ]:
df = df[df.gender != 'Other']
df.gender.value_counts()

#### Hay que tener en consideración la pérdida de datos, hemos pasado de 5110 registros a 3566. Sin embargo, estos datos siguen siendo válidos, más limpios y pueden ser utilizados para el modelo de ML. 

#### Llegado a este punto, el data set pasa a estar a un estado mucho más útil, limpio y con mayor facilidad para su manipulación.  
#### Comenzaremos con el EDA para obtener respuestas, patrones y entender mejor los datos

#### Emplearemos el uso de un mapa de correlación

In [ ]:
#Creamos un df sin la columna id para ser utizado en el mapa de correlación, ya que el mismo es un identificador único.
dfHeatmap = df.drop(columns=['id'])
plt.figure(dpi = 120,figsize= (6,6))
mask = np.triu(np.ones_like(dfHeatmap.corr(),dtype = bool))
sns.heatmap(dfHeatmap.corr(),mask = mask, fmt = ".2f",annot=True,lw=1,cmap = 'plasma', vmax=1, vmin=0)
plt.yticks(rotation = 0)
plt.xticks(rotation = 90)
plt.title('Correlation Heatmap')
plt.show()


Se ha decidido descartar la columna residence_type, ya que esta no presenta un impacto en el dataset ni correlaciones positivas y/o negativas de forma significante. (-0,02 como el mayor coeficiente)

In [ ]:
#Eliminamos la columna residence_type antes de seguir analizando en profundida el dataset.
df.drop(['residence_type'], axis=1, inplace=True)
df.info()

Se ha decidido eliminar la columna ever_married, ya que si bien es la variable más correlacionada con age, esta sólo nos demuestra que a mayor edad, la gente es más propensa a estar casada. Este dato no nos aporta valor para el objetivo al que queremos llegar. Además, se han hecho pruebas con los modelos y este valor no tiene influencia en el rendimiento de los mismos.

In [ ]:
df.drop(['ever_married'], axis=1, inplace=True)
df.info()

#### Observaremos la distribución de estado de fumador en base a la edad y como esta influye a la hora de padecer un ACV

In [ ]:
df_smoking = df[(df['smoking_status']) != 'Unknown']
df_smoking2 = df.copy()
df_smoking2['stroke'].replace(
                to_replace=[0, 1], 
                value=["No","Yes"], 
                inplace=True)
ax = sns.catplot(data=df_smoking2, kind='violin', x='smoking_status', y='age',   hue='stroke', split=True)
ax.set(xlabel='Smoking Status', ylabel='Edad', title='Categorización de ACV por edad y estado de fumador.')
plt.show()

#### A continuación realizaremos gráficos de boxplot para todas las columnas, con el fin de observar con mayor detalle la distribución de valores junto a la presencia (o no) de outliers.

In [ ]:
variables = ['bmi', 'avg_glucose_level', 'age']

fig, axes = plt.subplots(1, len(variables), figsize=(10,5))

for ax, variable in zip(axes, variables):
    ax = sns.boxplot( y=variable, data=df, ax=ax)
plt.tight_layout()

plt.show()

#### En los gráficos de boxplot se pueden observar una gran presencia de outliers (valores atípicos), estos se mantendrán en el dataset ya que en este caso en particular, no presenta un problema, siendo que son valores válidos y representan la gran cantidad de datos. Eliminarlos reduciría la utilidad del dataset de forma drástica

#### Aplicación de modelos de ML

#### Una vez realizado los diferentes tipos de análisis y limpieza de datos necesaria, es momento de aplicar algoritmos.

In [ ]:
#Creamos un data dummie para obtener las variables categóricas como booleanas (es decir 0,1)
df_dummy = pd.get_dummies(df, drop_first=True)
df_dummy = df_dummy.drop('id', axis=1)
df_dummy    

In [ ]:
#Separamos los datos de entrada de la salida
X = df_dummy.drop('stroke', axis=1) #Elimino de mi dataset la variable a predecir
y = df_dummy.stroke #Defino el Target

In [ ]:
#Seteamos los valores para separar el testeo del train, un 20% para testear y 80% para entrenamiento.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42) 

##### Como primer algoritmo, nos decantaremos por la regresión logística binaria en base a su naturaleza de predecir la probabilidad de una variable categórica. (En este caso, 1 sí es propenso a tener un ACV; 0 si no)

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [ ]:
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

#### Para continuar, el segundo algoritmo que utilizaremos sera el DecisionTreeClassifier (Arbol de decisión)

In [ ]:
tree = DecisionTreeClassifier(random_state = 42, class_weight="balanced") #Creamos el modelo

#### Se agregó el hiperparámetro class_weight="balanced" para ayudar al algoritmo DecisionTree a trabajar con un set de datos desbalanceado.

In [ ]:
param_dict = {
    "criterion":['gini','entropy'],
    "max_depth":range(1,10),
    "min_samples_split":range(1,10),
    "min_samples_leaf":range(1,5)
}

In [ ]:
grid = GridSearchCV(tree,
                    param_grid=param_dict,
                    cv=3,
                    verbose=1,
                    n_jobs=1)
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_score_

In [ ]:
y_train_pred = tree.predict(X_train) #Prediccion en Train
y_test_pred = tree.predict(X_test) #Prediccion en Test

In [ ]:
class_probabilities = grid.predict_proba(X_test)
preds = class_probabilities[:, 1]

fpr, tpr, threshold = roc_curve(y_test, preds)
roc_auc = auc(fpr, tpr)

# AUC
print(f"AUC for our classifier is: {roc_auc}")

# Gráfica de la Curva ROC
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
#Matriz de confusión para evaluar el desempeño del algoritmo
print(confusion_matrix(y_test, y_test_pred))

#### Además del algoritmo DecisionTree, aplicaremos KNeighborsClassifier y Random Forest para ver su efectividad.

In [ ]:
#Creamos nuestro objeto KNN
knn = KNeighborsClassifier()

#### Recurriremos al uso de GridSearchCV para obtener la mejor configuración de hiper parámetros.

In [ ]:
#Definicion de Hyperparámetros
param_grid = {'n_neighbors':np.arange(1, 10),
              'weights': ['uniform', 'distance'], 
              'leaf_size':[1,3,5,7,10],
              'algorithm':['auto', 'kd_tree']}

#Utilizamos la grilla definida anteriormente...
model = GridSearchCV(knn, param_grid=param_grid, cv=3)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
print("Mejores parametros: "+str(model.best_params_))
print("Mejor Score: "+str(model.best_score_)+'\n')

In [ ]:
scores = pd.DataFrame(model.cv_results_)
scores

In [ ]:
prediction = model.predict(X_test)

In [ ]:
#Accuracy
print('Exactitud:', accuracy_score(y_test, prediction))

In [ ]:
class_probabilities = model.predict_proba(X_test)
preds = class_probabilities[:, 1]

fpr, tpr, threshold = roc_curve(y_test, preds)
roc_auc = auc(fpr, tpr)

# AUC
print(f"AUC for our classifier is: {roc_auc}")

# Gráfica de la Curva ROC
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
#Acudiremos a una matriz de confusión para obtener los resultados del modelo
print(confusion_matrix(y_test, y_test_pred))

#Ploteamos la Matriz

plot_confusion_matrix(model, X_test, y_test)

plt.show()

In [ ]:
#Creamos un random forest!
rfc=RandomForestClassifier(random_state=42, class_weight="balanced")
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}


In [ ]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 3)
CV_rfc.fit(X_train, y_train)    

In [ ]:
CV_rfc.best_params_

In [ ]:
rfc1=RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 200, max_depth=2, criterion='gini')

In [ ]:
rfc1.fit(X_train, y_train)

In [ ]:
pred=rfc1.predict(X_test)

In [ ]:
y_test_pred = model.predict(X_test) #Prediccion en Test

In [ ]:
class_probabilities = rfc1.predict_proba(X_test)
preds = class_probabilities[:, 1]

fpr, tpr, threshold = roc_curve(y_test, preds)
roc_auc = auc(fpr, tpr)

# AUC
print(f"AUC for our classifier is: {roc_auc}")

# Gráfica de la Curva ROC
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
#Matriz de confusión para evaluar el desempeño del algoritmo RANDOM FOREST
print(confusion_matrix(y_test, y_test_pred))
plot_confusion_matrix(rfc1, X_test, y_test)
plt.show()